In [2]:
import pandas as pd
import numpy as np
import sklearn
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhiti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
df1=pd.read_excel(r"datasets/extremism.xlsx")

df=pd.DataFrame()
df['id'] = df1['id']
df['text'] = df1['text']
df['class'] = df1['class']
df = df.dropna()

In [38]:
df

,id,text,class
0,1,Таким образом никто не внес большего вклад...,3.0
1,2,"Можно с уверенностью сказать, что все полезное...",3.0
2,3,Славяне должны на нас работать. Если они нам б...,3.0
3,4,"Русское государство росло, развивалось из свои...",3.0
4,5,Поставим власть под контроль народа,3.0
...,...,...,...
335,336,NaN,NaN
336,337,NaN,NaN
337,338,NaN,NaN
338,339,NaN,NaN


In [27]:
df.dropna(axis=0,inplace=True)

In [31]:
def comma2space(word):
    new_word = ''
    try:
        for char in word:
            if char in patterns:
                char = ' '
            new_word += char
    except TypeError:
        pass
    return new_word
    
def lemmatize(doc):
    doc = re.sub(patterns, '', doc)
    tokens = []
    for token in doc.split():
        if token:
            
            token = token.lower()
            token = token.strip()
            tokens.append(token)
    return tokens

def dropstopwords(docs):
    stopwords = nltk.corpus.stopwords.words('russian')
    sen =  []
    for doc in docs:
        if (doc not in stopwords):
            sen.append(doc)
    return sen

def dropcomma(doc):
    new_sen = ''
    for char in doc:
        if (char not in patterns):
            new_sen += char
    return new_sen
def tokenize(docs):
    docs = comma2space(docs)
    docs = lemmatize(docs)
    docs = dropstopwords(docs)
    return docs

In [46]:
data=df['text'].apply(tokenize)
data

14     [убивай, хача, мочи, хача, бей, черных, бей, х...
59            [россия, русских, долой, инородную, мразь]
92     [аллах, даруй, нам, стойкости, пока, увидим, н...
116    [твою, боль, собой, заберу, могилу, твои, печа...
197    [самоубийцы, считай, телесной, боли, руки, нак...
                             ...                        
178                         [такую, погоду, повеситься…]
181    [никому, позволял, коснуться, большей, части, ...
156    [лесу, веселых, трупов, мертвецки, пьян, брел,...
146    [терять, страшно, заднем, фоне, взорвалась, мы...
131    [думали, смерти, забыли, такое, счастье, верно...
Name: text, Length: 201, dtype: object

In [80]:
import time
import multiprocessing
#WORD2VEC()
from gensim.models import Word2Vec
w2v_model = Word2Vec(
    min_count=5,
    window=4,
    size=30,
    negative=10,
    alpha=0.5,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1,workers=4)

#BUILD_VOCAB()
t = time.time()
w2v_model.build_vocab(data, progress_per=100)
print("Time to build vocab': {} mins".format(round((time.time() - t) / 60, 2)))

#TRAIN()
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=300, report_delay=1)
print('Time to train the model: {} mins'.format(round((time.time() - t) / 60, 2)))

Time to build vocab': 0.0 mins
Time to train the model: 0.01 mins


In [81]:
w2v_model.init_sims(replace=True)
w2v_model.wv.save_word2vec_format('hachi.bin', binary=True)

In [82]:
model1 = w2v_model.wv.load_word2vec_format(r'C:\Users\Zero\exp_model.bin', binary=True)
model1.syn0

<ipython-input-82-c8ac0a104979>:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  model1.syn0


array([[ 0.01329773, -0.04954648,  0.07695488, ...,  0.07561335,
         0.12918606, -0.00850406],
       [ 0.011624  , -0.01226313,  0.01542811, ...,  0.07488082,
        -0.05080352,  0.05965197],
       [-0.05951629, -0.06781738,  0.01338605, ...,  0.07335977,
         0.12016334,  0.05530556],
       ...,
       [ 0.01125513,  0.02318161,  0.02822547, ...,  0.03792449,
         0.00100433,  0.0697925 ],
       [ 0.03417982, -0.04482038,  0.07471242, ...,  0.0877353 ,
         0.02725162,  0.05327835],
       [-0.01022633, -0.04199966,  0.05124441, ...,  0.09970852,
         0.02095896,  0.0199973 ]], dtype=float32)

In [87]:
model1.most_similar(positive=["смерть"])

[('короткая', 0.8679245114326477),
 ('любая', 0.8675863146781921),
 ('светлая', 0.8640148639678955),
 ('идиотская', 0.8541393280029297),
 ('нравилась', 0.8533639311790466),
 ('душевная', 0.849618673324585),
 ('великолепная', 0.8489678502082825),
 ('лапочка', 0.8488980531692505),
 ('удивительная', 0.847754716873169),
 ('важная', 0.8469743728637695)]

In [88]:
import numpy as np
from scipy import spatial

index2word_set = set(model1.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

<ipython-input-88-37b38b84b612>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  index2word_set = set(model1.wv.index2word)


In [102]:
listok=['прыгунть в окно','разбежаться в пропасть',"валить пидоров"]
deist='хочу умереть'
score=[]
slovar={}
for i in range(len(listok)):
    s1_afv = avg_feature_vector(deist, model=model1, num_features=300, index2word_set=index2word_set)
    s2_afv = avg_feature_vector(listok[i], model=model1, num_features=300, index2word_set=index2word_set)
    sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
    score.append(sim)
    slovar.update({deist+' / '+listok[i]:sim})
sorted_tuples = sorted(slovar.items(), key=lambda item: item[1])

sorted_dict = {k: v for k, v in sorted_tuples}

print(sorted_dict)  # {1: 1, 3: 4, 2: 9}

{'хочу умереть / прыгунть в окно': 0.4668493866920471, 'хочу умереть / валить пидоров': 0.64153653383255, 'хочу умереть / разбежаться в пропасть': 0.6599088311195374}
